<a href="https://www.kaggle.com/code/zahrasyifaul/ascend-hore?scriptVersionId=247626883" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
# impor data
train_df = pd.read_csv('/kaggle/input/seleksi-dsa-compfest-17/train.csv')
test_df = pd.read_csv('/kaggle/input/seleksi-dsa-compfest-17/test.csv')

train_df.head(), test_df.head()

(                     ID        date cluster_id  electricity_consumption  \
 0  cluster_1_2014-01-01  2014-01-01  cluster_1                  358.032   
 1  cluster_2_2014-01-01  2014-01-01  cluster_2                  548.247   
 2  cluster_3_2014-01-01  2014-01-01  cluster_3                  758.303   
 3  cluster_4_2014-01-01  2014-01-01  cluster_4                 1072.077   
 4  cluster_1_2014-01-02  2014-01-02  cluster_1                  386.908   
 
    temperature_2m_max  temperature_2m_min  apparent_temperature_max  \
 0                10.8                 4.2                       5.5   
 1                12.2                 4.3                       8.6   
 2                12.9                -0.8                      10.1   
 3                10.8                 4.7                       6.7   
 4                10.7                 7.0                       6.6   
 
    apparent_temperature_min  sunshine_duration  daylight_duration  \
 0                       0.4          

In [3]:
X_train = train_df.drop(columns=['ID', 'cluster_id', 'electricity_consumption', 'date'])
y_train = train_df['electricity_consumption']
X_test = test_df.drop(columns=['ID', 'cluster_id', 'date'])

In [4]:
# ngapus kolom 'date', 'ID', dan 'cluster_id'
train_features = train_df.drop(['ID', 'cluster_id', 'electricity_consumption', 'date'], axis=1)
test_features = test_df.drop(['ID', 'cluster_id', 'date'], axis=1)

# tampilin beberapa baris data untuk memastikan perubahan
train_features.head(), test_features.head()

(   temperature_2m_max  temperature_2m_min  apparent_temperature_max  \
 0                10.8                 4.2                       5.5   
 1                12.2                 4.3                       8.6   
 2                12.9                -0.8                      10.1   
 3                10.8                 4.7                       6.7   
 4                10.7                 7.0                       6.6   
 
    apparent_temperature_min  sunshine_duration  daylight_duration  \
 0                       0.4          53.003333       29787.533333   
 1                      -0.4        8195.656667       30650.350000   
 2                      -4.9       16305.260000       31547.686667   
 3                       0.6        9224.803333       30769.220000   
 4                       3.3       22372.000000       29850.226667   
 
    wind_speed_10m_max  wind_gusts_10m_max  wind_direction_10m_dominant  \
 0                40.6                79.2                   186.0000

In [5]:
train_target = train_df['electricity_consumption']


In [6]:
#linear regression
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
pred_lr = model_lr.predict(X_test)

In [7]:
#random forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)

In [8]:
#combine
pred_combined = (pred_lr + pred_rf) / 2

In [9]:
# bagi data test jd 3 bagian
test_subset_1 = test_df.iloc[:1133]  # 1/3 pertama dari 3400
test_subset_2 = test_df.iloc[1133:2266]  # 1/3 kedua
test_subset_3 = test_df.iloc[2266:]  # 1/3 ketiga

# tampilin jumlah baris utk tiap subset untuk pastiin pembagian yang tepat
print(test_subset_1.shape, test_subset_2.shape, test_subset_3.shape)

(1133, 14) (1133, 14) (1134, 14)


In [10]:
# Buat fitur untuk subset pertama
test_features_1 = test_subset_1.drop(['ID', 'cluster_id', 'date'], axis=1)
predictions_1 = model_rf.predict(test_features_1)

# Buat fitur untuk subset kedua
test_features_2 = test_subset_2.drop(['ID', 'cluster_id', 'date'], axis=1)
predictions_2 = model_rf.predict(test_features_2)

# Buat fitur untuk subset ketiga
test_features_3 = test_subset_3.drop(['ID', 'cluster_id', 'date'], axis=1)
predictions_3 = model_rf.predict(test_features_3)

# Gabungkan semua prediksi
predictions_all = np.concatenate([predictions_1, predictions_2, predictions_3])

# Tampilkan jumlah prediksi
print(f"Jumlah total prediksi: {predictions_all.shape[0]}")

Jumlah total prediksi: 3400


In [11]:
submission = test_df[['ID']].copy()
submission['electricity_consumption'] = pred_combined
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [12]:
print(submission.head())

                     ID  electricity_consumption
0  cluster_1_2022-01-01               472.520235
1  cluster_2_2022-01-01               679.596586
2  cluster_3_2022-01-01               694.936246
3  cluster_4_2022-01-01               790.514729
4  cluster_1_2022-01-02               499.554957


In [13]:
# ==== [1] FEATURE ENGINEERING ====
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Baca data
df = pd.read_csv("/kaggle/input/seleksi-dsa-compfest-17/train.csv")


print(df.columns.tolist())


# 2. Cek kolom yang tersedia (penting untuk melihat nama kolom target yang benar)
print(df.columns)

# 3. Ubah kolom 'date' jadi datetime
df['date'] = pd.to_datetime(df['date'])

# 4. Tambahkan fitur waktu baru
df['dayofweek'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day'] = df['date'].dt.day
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

# 5. Misal target kamu bukan 'target', ubah jadi nama kolom yang benar (misalnya: 'jumlah')
# Ganti 'jumlah' sesuai hasil print(df.columns)
y = df['electricity_consumption']
X = df.drop(columns=['ID', 'date', 'electricity_consumption'])

# 2. Encode kolom kategorikal (cluster_id)
le = LabelEncoder()
X['cluster_id'] = le.fit_transform(X['cluster_id'])

# ==== [2] DATA SCALING ====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

['ID', 'date', 'cluster_id', 'electricity_consumption', 'temperature_2m_max', 'temperature_2m_min', 'apparent_temperature_max', 'apparent_temperature_min', 'sunshine_duration', 'daylight_duration', 'wind_speed_10m_max', 'wind_gusts_10m_max', 'wind_direction_10m_dominant', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']
Index(['ID', 'date', 'cluster_id', 'electricity_consumption',
       'temperature_2m_max', 'temperature_2m_min', 'apparent_temperature_max',
       'apparent_temperature_min', 'sunshine_duration', 'daylight_duration',
       'wind_speed_10m_max', 'wind_gusts_10m_max',
       'wind_direction_10m_dominant', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration'],
      dtype='object')


In [14]:
# ==== [3] MODEL LAIN ====
from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# Definisikan beberapa model
model_gbr = GradientBoostingRegressor()
model_xgb = XGBRegressor()
model_cat = CatBoostRegressor(verbose=0)
model_lgb = LGBMRegressor()

In [15]:
# ==== [4] HYPERPARAMETER TUNING ====
from sklearn.model_selection import GridSearchCV

# Contoh tuning untuk GradientBoosting
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7]
}
grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [16]:
# ==== [5] ENSEMBLING ====
ensemble = VotingRegressor([
    ('xgb', model_xgb),
    ('cat', model_cat),
    ('lgb', model_lgb)
])
ensemble.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000654 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2783
[LightGBM] [Info] Number of data points in the train set: 11688, number of used features: 11
[LightGBM] [Info] Start training from score 642.313328


VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate...n=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x7b5966ebf0d0>),
                            ('lgb', LGBMRegressor())])